In [2]:
import requests
from bs4 import BeautifulSoup
import time

## 預計抓取資料
- 飯店名稱
- 房客評價平均星等
- 房客評論數
- 房客留言內容
- 參考價格(爬蟲爬得時候的價格)
- 相關訂房網站的價格
- 飯店地址
- 飯店(推薦的)ranking (tripadviosr給的)

---
可能可以抓的：
- 飯店星等
- 飯店設備(游泳池之類)
- 是否有供餐
- 是否提供wifi
- 是否有車位
- 有沒有room service
- 訂房後能不能免費取消
- 是否入住後才需要付款(不用網站上給訂金)

### 取飯店列表

In [9]:
url = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

def get_hotel(url):
    #每次调用等待两秒, 避免被ban ip
    time.sleep(2)
    
    uri_list = []
    
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'html.parser')

    #imgs = soup.select('div.aspect.is-hidden-tablet > div.inner')
    titles = soup.select('div.listing_title > a[target="_blank"]')
    paimings = soup.select('div.popindex') # 排名
    prices = soup.select('div[data-sizegroup="mini-meta-price"]')
    agoda = soup.select('div[title="Agoda.com"]')
    hotel_com = soup.select('div[title="Hotels.com"]')
    
#     print(prices[0].parent.parent.parent.parent.find_next_siblings('div')[0].select('div[title="Agoda.com"]')[0].find_next_siblings('div')[0].get_text())
    
    # 取其他平台的價格
#     print(agoda[0].find_next_siblings('div')[0].get_text())
#     print(hotel_com[0].find_next_siblings('div'))

    data_list = []

    # https://www.saltycrane.com/blog/2008/04/how-to-use-pythons-enumerate-and-zip-to/
    for i, (title,paiming,price) in enumerate(zip(titles,paimings,prices)):
        # 放肯定會有的資料
        data = {
            'title':title.get_text(),
            'paiming':paiming.get_text(),
            'recommend_price':price.get_text(), # tripadviosr推薦的訂房網站價格，不一定最低
            'uri': 'https://www.tripadvisor.com.tw' + title.get('href')
        }
        
        # 根據uri抓飯店細節
        per_hotel_json = json.loads(get_hotel_detail('https://www.tripadvisor.com.tw' + title.get('href')))
        data['hotel_address'] = per_hotel_json['address']['streetAddress']
        data['avg_rating'] = per_hotel_json['aggregateRating']['ratingValue']
        data['comment_count'] = per_hotel_json['aggregateRating']['reviewCount']
        data['offical_img_uri'] = per_hotel_json['image']
        
        # 抓飯店關於service的內容
        json_tmp = json.loads(json.dumps(get_services(data['uri']))) # list先轉成string再轉json
        facility_list = []
        property_list = []
        for m in json_tmp:
            # 找跟飯店設備有關的訊息
            if json.loads(m)['export'] is None:
                # 房間設備
                for n in json.loads(m)['subprops']['amenities']['highlightedAmenities']['roomAmenities']:
                    facility_list.append(n['amenityNameLocalized'])
                data['facility'] = facility_list
                
                # 飯店設備
                for w in json.loads(m)['subprops']['amenities']['highlightedAmenities']['propertyAmenities']:
                    property_list.append(w['amenityNameLocalized'])
                data['property'] = property_list
        
        get_agoda = prices[i].parent.parent.parent.parent.find_next_siblings('div')[0].select('div[title="Agoda.com"]')
        if(get_agoda):
            # agoda有可能不在其他網站名單內
            data['agoda_price'] = get_agoda[0].find_next_siblings('div')[0].get_text()
        
#         print(data)
        
        uri_list.append('https://www.tripadvisor.com.tw' + title.get('href'))
        data_list.append(data)
    
    return uri_list, data_list

單一家飯店

In [10]:
all_uri, all_data = get_hotel(url)

這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：7
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：9
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：7
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：6
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0


自動換頁(去抓其他飯店)

In [17]:
url_list = ['https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html'.format(str(i)) for i in range(0,1200,30)]

In [18]:
all_data = []
for k in range(0,10):
    hotel_url, hotels_data = get_hotel(url_list[k])
    all_data = all_data + hotels_data

這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：7
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：9
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：1
這間飯店的第一頁評論數目：9
這間飯店的第一頁評論數目：6
這間飯店的第一頁評論數目：6
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：11
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：9
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：8
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：5
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：10
這間飯店的第一頁評論數目：0
這間飯店的第一頁評論數目：10
這

房間售完/ 已被網站排除推薦

In [18]:
hotel_url = get_hotel(url_list[38])

### 根據各別飯店取細節
- 地點, 清潔與衛生, 服務, 划算度的星等(房客評價)
- 飯店簡介
- 飯店縮圖連結
- 客人/ 飯店官方圖輯

In [5]:
import json

def get_hotel_detail(url):
    time.sleep(2)
    
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text,'html.parser')
    
    meta_data = soup.select('script[type="application/ld+json"]')
    
    # 使用者評論內容
    #     user_comment = soup.findAll("div", {"class": "common-text-ReadMore__content--2X4LR"})
    user_comment = soup.select('.common-text-ReadMore__content--2X4LR')
    print('這間飯店的第一頁評論數目：' + str(len(user_comment)))
    first_page_comment = []
#     if len(user_comment) > 0:
#         if user_comment[0].find('span'):
#         #     print(user_comment[0].text)
#             print(user_comment[0].q.span.text)
#         else:
#             print(user_comment[0].get_text())
    
    # 回傳飯店小細節的text(可以parser成json)
    return meta_data[0].get_text()

In [86]:
json.loads(get_hotel_detail(all_uri[0]))

<span>每年都會固定買旅展餐卷，這裡就像第二個家，菜色真的進步很多，每年來都有驚喜，甜點區，現榨果汁，牛排等等，都很不錯，希望能繼續保持。</span>


{'@context': 'http://schema.org',
 '@type': 'LodgingBusiness',
 'name': '台北花園大酒店',
 'url': '/Hotel_Review-g13808853-d1224203-Reviews-Taipei_Garden_Hotel-Zhongzheng_District_Taipei.html',
 'image': 'https://media-cdn.tripadvisor.com/media/photo-s/09/32/1f/6c/caption.jpg',
 'priceRange': 'NT$3,146 - NT$6,639 (根據標準客房的平均房價)',
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': '4.5',
  'reviewCount': '2750'},
 'address': {'@type': 'PostalAddress',
  'streetAddress': '中正區中華路二段1號',
  'addressLocality': '',
  'addressRegion': '',
  'postalCode': '10065',
  'addressCountry': {'@type': 'Country', 'name': '台灣'}}}

In [7]:
def get_services(url):
    time.sleep(2)
    
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text,'html.parser')
    
    service_data = soup.select('div[class="common-ssronly-CsrPortal__portal--2v9IC"]')
    
    service_data_list = []
    for data in service_data:
        service_data_list.append(data.get('data-csrprops'))
    
    return service_data_list

In [14]:
json_tmp = json.loads(json.dumps(get_services(all_uri[1])))
for m in json_tmp:
    if(json.loads(m)['export'] is None):
#         print(json.loads(m)['subprops']['amenities']['highlightedAmenities'])
#         print(len(json.loads(m)['subprops']['amenities']['highlightedAmenities']['roomAmenities']))
        for n in json.loads(m)['subprops']['amenities']['highlightedAmenities']:
            print(n)
#             print(n['amenityNameLocalized'])

{'roomAmenities': [{'tagId': 9184, 'amenityNameLocalized': '禁菸客房', 'amenityCategoryName': 'RoomTypes', 'amenityIcon': 'hotels', '__typename': 'Amenity'}, {'tagId': 9160, 'amenityNameLocalized': '套房', 'amenityCategoryName': 'RoomTypes', 'amenityIcon': 'hotels', '__typename': 'Amenity'}, {'tagId': 9659, 'amenityNameLocalized': '家庭房', 'amenityCategoryName': 'RoomTypes', 'amenityIcon': 'hotels', '__typename': 'Amenity'}, {'tagId': 9658, 'amenityNameLocalized': '空調設施', 'amenityCategoryName': 'RoomAmenities', 'amenityIcon': 'hotels', '__typename': 'Amenity'}, {'tagId': 10589, 'amenityNameLocalized': '保險箱', 'amenityCategoryName': 'RoomAmenities', 'amenityIcon': 'hotels', '__typename': 'Amenity'}, {'tagId': 9665, 'amenityNameLocalized': '冰箱', 'amenityCategoryName': 'KitchenAmenities', 'amenityIcon': 'restaurants', '__typename': 'Amenity'}, {'tagId': 11269, 'amenityNameLocalized': '平板電視', 'amenityCategoryName': 'Entertainment', 'amenityIcon': 'play', '__typename': 'Amenity'}], 'propertyAmenitie

### 整理成csv輸出
> https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe

In [19]:
import pandas as pd
# list of dictionaries to df
data_df = pd.DataFrame.from_dict(all_data)
data_df.head()

,title,paiming,recommend_price,uri,hotel_address,avg_rating,comment_count,offical_img_uri,facility,property,agoda_price
0,台北凱達大飯店,"台北排名第 1 的超值住宿地點，共 1,199 家","NT$2,261",https://www.tripadvisor.com.tw/Hotel_Review-g1...,艋舺大道167號,4.0,332,https://media-cdn.tripadvisor.com/media/photo-...,"[禁菸客房, 套房, 家庭房, 空調設施, 客房服務, 冰箱]","[免費高速上網 (WiFi), 游泳池, 有健身房的健身中心, 酒吧/酒廊, 機場接送, 提...","NT$2,251"
1,新驛旅店 - 復興北路店,"台北排名第 2 的超值住宿地點，共 1,199 家","NT$3,095",https://www.tripadvisor.com.tw/Hotel_Review-g1...,中山區復興北路338號,5.0,906,https://media-cdn.tripadvisor.com/media/photo-...,"[禁菸客房, 套房, 家庭房, 空調設施, 保險箱, 冰箱, 平板電視]","[免費停車, 停車, 免費高速上網 (WiFi), 機場接送, 提供上網服務的商務中心, 禁...",NaN
2,路境行旅,"台北排名第 3 的超值住宿地點，共 1,199 家","NT$2,601",https://www.tripadvisor.com.tw/Hotel_Review-g1...,中正區重慶南路一段86號7-11樓,4.5,173,https://media-cdn.tripadvisor.com/media/photo-...,NaN,NaN,NaN
3,台北美侖大飯店,"台北排名第 4 的超值住宿地點，共 1,199 家","NT$1,954",https://www.tripadvisor.com.tw/Hotel_Review-g1...,復興南路一段317號,4.5,788,https://media-cdn.tripadvisor.com/media/photo-...,"[禁菸客房, 套房, 家庭房, 空調設施, 迷你吧, 冰箱, 平板電視]","[免費停車, 免費高速上網 (WiFi), 有健身房的健身中心, 酒吧/酒廊, 提供上網服務...","NT$1,958"
4,老爺會館台北林森,"台北排名第 5 的超值住宿地點，共 1,199 家","NT$2,739",https://www.tripadvisor.com.tw/Hotel_Review-g1...,林森北路83號4樓 Near Huashan 1914 Creative Park,4.5,248,https://media-cdn.tripadvisor.com/media/photo-...,"[禁菸客房, 套房, 家庭房, 空調設施, 冰箱, 平板電視]","[免費高速上網 (WiFi), 免費網際網路, 無線網路, 免費早餐, 自助式早餐, 禁菸飯...",NaN


In [20]:
data_df.to_csv('./tapei_tripadvisor.csv', index=False, encoding='utf_8_sig')